In [281]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from pathlib import Path
import torch
from torch.utils.data import TensorDataset ,DataLoader
from torch import nn,optim
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/test.csv
/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv


**Loading Data**


In [282]:
train=pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test=pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
train.shape,test.shape

((42000, 785), (28000, 784))

**Extracting Input and Target Variable**

In [283]:
x=train.drop("label",axis=1)
y=np.array(train['label'])
x.shape,y.shape

((42000, 784), (42000,))

**Train -Test Split -Pytorch**

In [284]:
torch_X_train = torch.from_numpy(x.values).type(torch.FloatTensor)/255   # Normalizing the X values
torch_y_train = torch.from_numpy(y).type(torch.LongTensor)
myDataset = torch.utils.data.TensorDataset(torch_X_train,torch_y_train)
valid_no  = int(0.2 * len(myDataset))
# so divide the data into trainset and testset
trainSet,testSet = torch.utils.data.random_split(myDataset,(len(myDataset)-valid_no,valid_no))
print(f"len of trainSet {len(trainSet)} , len of testSet {len(testSet)}")
batch_size=64
trainloader  = DataLoader(trainSet , batch_size=batch_size ,shuffle=True) 
testloader  = DataLoader(testSet , batch_size=batch_size ,shuffle=True)

len of trainSet 33600 , len of testSet 8400


**Building Network**
Model 1

In [285]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 128)
        #self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)
        

        # Dropout module with 0.2 drop probability
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        # make sure input tensor is flattened
        x = x.view(x.shape[0], -1)

        # Now with dropout
        x = self.dropout(F.relu(self.fc1(x)))
        #x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))

        # output so no dropout here
        x = F.log_softmax(self.fc4(x), dim=1)

        return x
        
model=Network()

#Adam - adaptive learning rate optimization algorithm
optimizer=optim.Adam(model.parameters(),lr=0.001)


criterion=nn.NLLLoss()

In [286]:
epochs=8
train_losses,test_losses=[],[]
for e in range(epochs):
    running_loss=0
    for images,labels in trainloader:
        optimizer.zero_grad()
        #images=images.view(images.shape[0],-1)
        log_ps=model(images)
        loss=criterion(log_ps,labels) # a single value for ex 2.33
        loss.backward() 
        optimizer.step()
        running_loss += loss.item() * images.shape[0] ## (2.33*64 + 2.22*64 + 2.12*33) / 138 
        
    else:
        test_loss=0
        accuracy=0
        
        with torch.no_grad():
            model.eval()
            for images,labels in testloader:
                log_ps=model(images)
                test_loss+=criterion(log_ps,labels) *images.shape[0]
                ps=torch.exp(log_ps)
                top_p,top_class=ps.topk(1,dim=1)
                equals=top_class==labels.view(*top_class.shape)
                accuracy+=torch.sum(equals).item()
        model.train()
        train_losses.append(running_loss/len(trainloader.dataset))
        test_losses.append(test_loss.item()/len(testloader.dataset))

        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(running_loss/len(trainloader.dataset)),
              "Test Loss: {:.3f}.. ".format(test_loss/len(testloader.dataset)),
              "Test Accuracy: {:.3f}".format(accuracy/len(testloader.dataset)))

Epoch: 1/8..  Training Loss: 0.524..  Test Loss: 0.219..  Test Accuracy: 0.933
Epoch: 2/8..  Training Loss: 0.228..  Test Loss: 0.160..  Test Accuracy: 0.951
Epoch: 3/8..  Training Loss: 0.172..  Test Loss: 0.143..  Test Accuracy: 0.955
Epoch: 4/8..  Training Loss: 0.142..  Test Loss: 0.114..  Test Accuracy: 0.967
Epoch: 5/8..  Training Loss: 0.117..  Test Loss: 0.118..  Test Accuracy: 0.963
Epoch: 6/8..  Training Loss: 0.104..  Test Loss: 0.112..  Test Accuracy: 0.965
Epoch: 7/8..  Training Loss: 0.091..  Test Loss: 0.100..  Test Accuracy: 0.970
Epoch: 8/8..  Training Loss: 0.085..  Test Loss: 0.098..  Test Accuracy: 0.970


In [287]:
pd.DataFrame({"Predicted":top_class.view(top_class.shape[0]),"Actual":labels})

,Predicted,Actual
0,9,9
1,8,8
2,8,8
3,0,0
4,4,4
5,6,6
6,8,8
7,5,8
8,2,2
9,6,6


In [288]:
print("Best model: \n\n", model, '\n')
print("The state dict keys: \n\n", model.state_dict().keys())

# counting the number of parameters
pytorch_total_params = sum(p.numel() for p in model.parameters())
print(pytorch_total_params)

Best model: 

 Network(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=10, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
) 

The state dict keys: 

 odict_keys(['fc1.weight', 'fc1.bias', 'fc3.weight', 'fc3.bias', 'fc4.weight', 'fc4.bias'])
109386


Model 2

In [290]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 128)
        #self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)
        

        # Dropout module with 0.2 drop probability
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        # make sure input tensor is flattened
        x = x.view(x.shape[0], -1)

        # Now with dropout
        x = self.dropout(F.relu(self.fc1(x)))
        #x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))

        # output so no dropout here
        x = F.log_softmax(self.fc4(x), dim=1)

        return x
        
model=Network()


optimizer=optim.SGD(model.parameters(),lr=1e-2,weight_decay=1e-6, momentum=0.9, nesterov=True)


criterion=nn.NLLLoss()

In [293]:
epochs=8
train_losses,test_losses=[],[]
for e in range(epochs):
    running_loss=0
    for images,labels in trainloader:
        optimizer.zero_grad()
        #images=images.view(images.shape[0],-1)
        log_ps=model(images)
        loss=criterion(log_ps,labels) # a single value for ex 2.33
        loss.backward() 
        optimizer.step()
        running_loss += loss.item() * images.shape[0] ## (2.33*64 + 2.22*64 + 2.12*33) / 138 
        
    else:
        test_loss=0
        accuracy=0
        
        with torch.no_grad():
            model.eval()
            for images,labels in testloader:
                log_ps=model(images)
                test_loss+=criterion(log_ps,labels) *images.shape[0]
                ps=torch.exp(log_ps)
                top_p,top_class=ps.topk(1,dim=1)
                equals=top_class==labels.view(*top_class.shape)
                accuracy+=torch.sum(equals).item()
        model.train()
        train_losses.append(running_loss/len(trainloader.dataset))
        test_losses.append(test_loss.item()/len(testloader.dataset))

        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(running_loss/len(trainloader.dataset)),
              "Test Loss: {:.3f}.. ".format(test_loss/len(testloader.dataset)),
              "Test Accuracy: {:.3f}".format(accuracy/len(testloader.dataset)))

Epoch: 1/8..  Training Loss: 0.063..  Test Loss: 0.087..  Test Accuracy: 0.973
Epoch: 2/8..  Training Loss: 0.061..  Test Loss: 0.087..  Test Accuracy: 0.974
Epoch: 3/8..  Training Loss: 0.056..  Test Loss: 0.086..  Test Accuracy: 0.975
Epoch: 4/8..  Training Loss: 0.056..  Test Loss: 0.082..  Test Accuracy: 0.976
Epoch: 5/8..  Training Loss: 0.053..  Test Loss: 0.083..  Test Accuracy: 0.975
Epoch: 6/8..  Training Loss: 0.052..  Test Loss: 0.080..  Test Accuracy: 0.976
Epoch: 7/8..  Training Loss: 0.047..  Test Loss: 0.081..  Test Accuracy: 0.976
Epoch: 8/8..  Training Loss: 0.046..  Test Loss: 0.085..  Test Accuracy: 0.976


In [294]:
print("Best model: \n\n", model, '\n')
print("The state dict keys: \n\n", model.state_dict().keys())

# counting the number of parameters
pytorch_total_params = sum(p.numel() for p in model.parameters())
print(pytorch_total_params)

Best model: 

 Network(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=10, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
) 

The state dict keys: 

 odict_keys(['fc1.weight', 'fc1.bias', 'fc3.weight', 'fc3.bias', 'fc4.weight', 'fc4.bias'])
109386


In [ ]:
torch.save(model.state_dict(), 'checkpoint.pth')

In [ ]:
test_images = pd.read_csv("../input/digit-recognizer/test.csv")
test_image = test_images.loc[:,test_images.columns != "label"].values
test_dataset = torch.from_numpy(test_image).type(torch.FloatTensor)/255
print(test_dataset.shape)
#test_dataset = torch.utils.data.TensorDataset(test_dataset)
new_test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 100, shuffle = False)

In [ ]:
results = []
with torch.no_grad():
    model.eval()
    for images in new_test_loader:
        output = model(images)
        ps = torch.exp(output)
        top_p, top_class = ps.topk(1, dim = 1)
        results += top_class.numpy().tolist()

In [ ]:
predictions = np.array(results).flatten()
print(predictions[:5])
print(predictions.shape)

In [ ]:
submissions=pd.DataFrame({"ImageId": list(range(1,len(predictions)+1)),
                         "Label": predictions})
submissions.to_csv("my_submissions.csv", index=False, header=True)

Model 1: 97%
Architecture description:

1. Batch Size: 64
2. Learning rate: 0.001
3. Number of parameters: 109386
4. Dropout: 0.2
5. Optimizer: Adam
6. Input layer: 784
7. Output layer: 10
8. Number of epochs: 8

Model 2: 97.6%
Architecture description:

1. Batch Size: 64
2. Learning rate: 0.001
3. Number of parameters: 109386
4. Dropout: 0.2
5. Optimizer: SGD(lr=1e-2,decay=1e-6, momentum=0.9, nesterov=True)
6. Input layer: 784
7. Output layer: 10
8. Number of epochs: 8